In [2]:
import zipfile
import os

# 1. Preprocessing Data

## 1a. Extract from archive

In [4]:
# extract data from archive

zip_file = "data/raw/archive.zip"
extract_dir = "data/raw/"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

## 1b. Preprocessing Data